In [1]:
import pandas as pd
import glob
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
os.environ["SCIPY_ARRAY_API"] = "1"

In [2]:
# путь к папке
folder = "data"

# ищем все csv-файлы
all_csv = glob.glob(os.path.join(folder, "*.csv"))

# читаем и объединяем
df_0 = pd.concat((pd.read_csv(f) for f in all_csv), ignore_index=True)
print(f"Считано файлов: {len(all_csv)}")
# переименуем для удобства
df_0 = df_0.rename(columns={"BBLS_OIL_COND":"oil", "MCF_GAS": 'gas', "BBLS_WTR":"water", "API_WellNo":"well_name", "RptDate":"date", "DAYS_PROD":"days_prod"})
# преобразуем колонку RptDate к datetime
df_0["date"] = pd.to_datetime(df_0["date"])

Считано файлов: 4485


In [3]:
df = df_0.drop(columns=["Lease_Unit", "Formation"])
print(df.shape)

(1212779, 6)


In [4]:
# важно, чтобы внутри каждой скважины ряды шли строго по времени
df = df.sort_values(by=["well_name", "date"]).reset_index(drop=True)

In [5]:
# Есть записи с отрицательными дебитами. Исключим их, так как работа будет вестись только с добывающими скважинами
df = df[(df['oil'] >= 0) & (df['gas'] >= 0) & (df['water'] >= 0)]

In [6]:
df

,well_name,date,oil,gas,water,days_prod
0,25-003-05000-00-00,1991-02-28,0,0,0,0.0
1,25-003-05001-00-00,1991-01-31,0,0,0,0.0
2,25-003-05001-00-00,1991-03-31,0,0,0,0.0
3,25-003-05001-00-00,1991-04-30,0,0,0,0.0
4,25-003-05001-00-00,1991-05-31,0,0,0,0.0
...,...,...,...,...,...,...
1212774,25-111-21271-00-00,2025-03-31,0,0,0,0.0
1212775,25-111-21271-00-00,2025-04-30,0,0,0,0.0
1212776,25-111-21271-00-00,2025-05-31,60,16,5505,16.0
1212777,25-111-21271-00-00,2025-06-30,0,0,0,0.0


In [7]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Предобработка профилей (Шаг 1)
------------------------------------------------------------------
Этот блок определяет конфиг и функции предобработки для датафрейма df
со столбцами:
['well_name', 'date', 'oil', 'water', 'gas', 'days_prod']

Основные шаги:
- Приведение к среднесуточным дебитам (учёт days_prod, иначе — дни в месяце)
- Единая месячная сетка и агрегация
- Фильтры качества и робастная обработка выбросов
- Сглаживание (Savitzky–Golay)
- Производные каналы (wc, gor), нормализация по пику
- Выравнивание времени (t=0 — старт производства)
- Усечение/паддинг до горизонта T
- Подготовка длинного и «tensor» представления

Код написан так, чтобы безопасно выполняться даже без df в окружении.
"""
from __future__ import annotations

import math
import warnings
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy.signal import savgol_filter


# ----------------------------
# Конфигурация предобработки
# ----------------------------

@dataclass
class PreprocConfig:
    # Временная сетка
    freq: str = "MS"                 # ежемесячно: начало месяца
    T: int = 70                      # целевой горизонт в месяцах (паддинг/обрезка)
    min_profile_months: int = 12     # минимальная длина валидного профиля (по основному каналу)

    # Порог старта производства для выравнивания (t=0)
    # Абс. порог для r_oil_s (сглаженный среднесуточный дебит нефти)
    prod_threshold_abs: float = 0.01
    # Относительный порог от пика r_oil_s (например, 5% от пика)
    prod_threshold_rel: float = 0.01

    # Сглаживание (Savitzky–Golay)
    smooth_window: int = 5           # должно быть нечётным, корректируется автоматически
    smooth_poly: int = 2

    # Робастная «подрезка» выбросов (winsorize) по КАЖДОЙ скважине
    winsorize_low: float = 0.01      # перцентиль снизу
    winsorize_high: float = 0.99     # перцентиль сверху

    # Нормализация амплитуды (деление на пик r_oil_s)
    normalize_by_peak: bool = True
    eps: float = 1e-9

    # Ограничения и заполнение
    clamp_nonnegative: bool = True   # отрицательные объёмы/дебиты → NaN (позже можно 0)
    interpolate_gaps: bool = True    # интерполяция пропусков для сглаживания

    # Каналы, которые пойдут в итоговый тензор (в таком порядке)
    tensor_channels: Tuple[str, ...] = (
        "r_oil_norm", "wc", "gor", "dr_oil_norm"
    )

    # Основной канал, по которому считаем длину валидного профиля
    primary_channel: str = "r_oil_s"


# ----------------------------
# Утилиты
# ----------------------------

def _safe_quantiles(s: pd.Series, ql: float, qh: float) -> Tuple[float, float]:
    s_clean = s[np.isfinite(s)]
    if s_clean.empty:
        return (np.nan, np.nan)
    return (s_clean.quantile(ql), s_clean.quantile(qh))


def _winsorize_per_well(s: pd.Series, low: float, high: float) -> pd.Series:
    if s.isna().all():
        return s
    ql, qh = _safe_quantiles(s, low, high)
    if not np.isfinite(ql) or not np.isfinite(qh):
        return s
    return s.clip(lower=ql, upper=qh)


def _savgol_safe(y: pd.Series, window: int, poly: int, do_interpolate: bool = True) -> pd.Series:
    if y.isna().all():
        return y
    x = y.copy()
    if do_interpolate:
        x = x.interpolate(limit_direction="both")
    # безопасная корректировка окна
    n = len(x)
    w = max(3, min(window, n))
    if w % 2 == 0:
        w = max(3, w - 1)
    p = min(poly, w - 1)
    if p < 1:
        # слишком короткий ряд — вернём как есть
        return y if not do_interpolate else x
    try:
        arr = savgol_filter(x.values.astype(float), window_length=w, polyorder=p, mode="interp")
        out = pd.Series(arr, index=y.index)
        return out
    except Exception:
        # на всякий случай резервация
        return y if not do_interpolate else x


def _month_start(ts: pd.Series) -> pd.Series:
    # Приводим даты к началу месяца
    return ts.dt.to_period("M").dt.to_timestamp()


def _days_in_month(dt: pd.Timestamp) -> int:
    # номинальные дни в месяце (если days_prod нет)
    return (dt + pd.offsets.MonthEnd(0) - (dt - pd.offsets.MonthBegin(1))).days + 1 - 1  # безопасно


def _weighted_rate(sum_volume: float, sum_days: float, eps: float) -> float:
    if not np.isfinite(sum_days) or sum_days <= 0:
        return np.nan
    return float(sum_volume) / (float(sum_days) + eps)


# ----------------------------------
# Основная функция предобработки
# ----------------------------------

def preprocess_profiles(
    df: pd.DataFrame,
    cfg: Optional[PreprocConfig] = None,
) -> Dict[str, object]:
    """
    Предобработка «сырых» данных до панели, выровненной по t=0 и ограниченной горизонтом T.
    Возвращает:
      - panel_long: long-таблица (well_name, t, date, каналы)
      - X: np.ndarray [n_wells, T, C] — тензор по cfg.tensor_channels
      - wells_used: список скважин, попавших в итог
      - dropped_summary: DataFrame со статистикой отбраковки
      - config: фактический конфиг (dict)

    Требуемые столбцы: well_name, date, oil, water, gas, days_prod
    """
    if cfg is None:
        cfg = PreprocConfig()

    required_cols = {"well_name", "date", "oil", "water", "gas", "days_prod"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Отсутствуют необходимые столбцы: {missing}")

    data = df.copy()

    # Типы
    data["well_name"] = data["well_name"].astype(str)
    data["date"] = pd.to_datetime(data["date"], errors="coerce")

    # Фильтрация строк с невалидной датой/именем
    data = data[ data["date"].notna() & data["well_name"].notna() ].copy()

    # Приведение отрицательных объёмов/дней к NaN (если включено)
    for col in ["oil", "water", "gas", "days_prod"]:
        if col in data.columns and cfg.clamp_nonnegative:
            data.loc[data[col] < 0, col] = np.nan

    # Приведение к началу месяца
    data["month"] = _month_start(data["date"])

    # Если days_prod отсутствует или нулевой — используем номинальные дни месяца
    # (это компромисс: лучше, чем делить на 30 фиксированно)
    # Параллельно аккумулируем объёмы по месяцу
    grp = data.groupby(["well_name", "month"], as_index=False).agg(
        oil_sum=("oil", "sum"),
        water_sum=("water", "sum"),
        gas_sum=("gas", "sum"),
        days_sum=("days_prod", "sum"),
        valid_days=("days_prod", lambda s: np.sum(np.isfinite(s) & (s > 0)))
    )

    # Фоллбек дней: если нет валидных days_prod за месяц, берём номинал
    # (важно: если исходные числа — уже среднесуточные, эту логику нужно отключить)
    # При необходимости пользователь может отдельно указать флаг в cfg, но для простоты — авто-детект.
    def _fallback_days(row):
        if row["valid_days"] and row["valid_days"] > 0:
            return row["days_sum"]
        return float(_days_in_month(pd.Timestamp(row["month"])))

    grp["days_eff"] = grp.apply(_fallback_days, axis=1)

    # Среднесуточные дебиты за месяц (взвешенное по дням)
    eps = cfg.eps
    grp["r_oil"] = grp.apply(lambda r: _weighted_rate(r["oil_sum"], r["days_eff"], eps), axis=1)
    grp["r_water"] = grp.apply(lambda r: _weighted_rate(r["water_sum"], r["days_eff"], eps), axis=1)
    grp["r_gas"] = grp.apply(lambda r: _weighted_rate(r["gas_sum"], r["days_eff"], eps), axis=1)

    # Сортировка
    grp = grp.sort_values(["well_name", "month"]).reset_index(drop=True)

    # По-скважинная обработка: winsorize → сглаживание → производные каналы → нормализация → выравнивание/усечение
    records: List[pd.DataFrame] = []
    dropped: List[Tuple[str, str]] = []  # (well_name, reason)

    for well, g in grp.groupby("well_name", sort=False):
        g = g.copy().reset_index(drop=True)

        # Робастная подрезка выбросов по КАЖДОМУ каналу дебитов
        for col in ["r_oil", "r_water", "r_gas"]:
            g[col] = _winsorize_per_well(g[col], cfg.winsorize_low, cfg.winsorize_high)

        # Интерполяция пропусков (опционально) — только для сглаживания
        do_interp = cfg.interpolate_gaps

        # Сглаживание
        g["r_oil_s"] = _savgol_safe(g["r_oil"], cfg.smooth_window, cfg.smooth_poly, do_interp)
        g["r_water_s"] = _savgol_safe(g["r_water"], cfg.smooth_window, cfg.smooth_poly, do_interp)
        g["r_gas_s"] = _savgol_safe(g["r_gas"], cfg.smooth_window, cfg.smooth_poly, do_interp)

        # Неотрицательность после сглаживания (если нужно)
        if cfg.clamp_nonnegative:
            for col in ["r_oil_s", "r_water_s", "r_gas_s"]:
                g.loc[g[col] < 0, col] = 0.0

        # Производные каналы
        g["wc"] = g["r_water_s"] / (g["r_oil_s"] + g["r_water_s"] + eps)
        g["gor"] = g["r_gas_s"] / (g["r_oil_s"] + eps)

        # Нормализация по пику r_oil_s
        if cfg.normalize_by_peak:
            peak = np.nanmax(g["r_oil_s"].values) if len(g) else np.nan
            scale = peak if (np.isfinite(peak) and peak > eps) else 1.0
            g["r_oil_norm"] = g["r_oil_s"] / (scale + eps)
            g["r_water_norm"] = g["r_water_s"] / (scale + eps)
            g["r_gas_norm"] = g["r_gas_s"] / (scale + eps)
        else:
            g["r_oil_norm"] = g["r_oil_s"]
            g["r_water_norm"] = g["r_water_s"]
            g["r_gas_norm"] = g["r_gas_s"]

        # Первая разность (динамика)
        g["dr_oil_norm"] = g["r_oil_norm"].diff()

        # Определяем точку старта t=0
        # Условие: r_oil_s >= max(prod_threshold_abs, prod_threshold_rel * peak)
        peak_s = np.nanmax(g["r_oil_s"].values) if len(g) else np.nan
        rel_thr = cfg.prod_threshold_rel * (peak_s if np.isfinite(peak_s) else 0.0)
        start_thr = max(cfg.prod_threshold_abs, rel_thr)
        start_idx = int(np.argmax(g["r_oil_s"].values >= start_thr)) if np.any(g["r_oil_s"].values >= start_thr) else None

        if start_idx is None:
            # Нет видимого старта — отбрасываем
            dropped.append((well, "no_start_detected"))
            continue

        g = g.iloc[start_idx:].copy().reset_index(drop=True)

        # Длина валидного профиля по основному каналу
        valid_len = int(np.sum(np.isfinite(g[cfg.primary_channel])))
        if valid_len < cfg.min_profile_months:
            dropped.append((well, f"too_short(<{cfg.min_profile_months})"))
            continue

        # Усечение/паддинг до T
        g = g.iloc[: cfg.T].copy()
        g["t"] = np.arange(len(g))
        # Паддинг — не добавляем строки с NaN, просто оставим длину < T;
        # при сборке тензора недостающие t будут NaN.

        # Сохраняем
        g["well_name"] = well
        records.append(g)

    if not records:
        panel_long = pd.DataFrame(columns=[
            "well_name", "t", "month",
            "r_oil", "r_water", "r_gas",
            "r_oil_s", "r_water_s", "r_gas_s",
            "wc", "gor",
            "r_oil_norm", "r_water_norm", "r_gas_norm",
            "dr_oil_norm"
        ])
        X = np.empty((0, cfg.T, len(cfg.tensor_channels)), dtype=float)
        wells_used: List[str] = []
    else:
        panel_long = pd.concat(records, ignore_index=True)
        panel_long = panel_long.rename(columns={"month": "date"})

        # Сборка тензора [n_wells, T, C] по cfg.tensor_channels
        channels = list(cfg.tensor_channels)
        wells_used = panel_long["well_name"].unique().tolist()
        n = len(wells_used)
        C = len(channels)
        T = cfg.T
        X = np.full((n, T, C), np.nan, dtype=float)

        # Быстрый доступ к индексам
        well_to_idx = {w: i for i, w in enumerate(wells_used)}
        ch_to_idx = {ch: j for j, ch in enumerate(channels)}

        for w, g in panel_long.groupby("well_name", sort=False):
            i = well_to_idx[w]
            # гарантируем t в пределах [0, T-1]
            g = g[(g["t"] >= 0) & (g["t"] < T)]
            for ch in channels:
                if ch not in g.columns:
                    continue
                X[i, g["t"].astype(int).values, ch_to_idx[ch]] = g[ch].values

    # Сводка по отбросам
    if dropped:
        dropped_df = pd.DataFrame(dropped, columns=["well_name", "reason"])
        dropped_summary = dropped_df["reason"].value_counts().rename_axis("reason").reset_index(name="count")
    else:
        dropped_summary = pd.DataFrame(columns=["reason", "count"])

    result = {
        "panel_long": panel_long,
        "X": X,
        "wells_used": wells_used,
        "dropped_summary": dropped_summary,
        "config": asdict(cfg),
        "tensor_channels": list(cfg.tensor_channels),
    }
    print("Preprocess complete.")
    print(f"  В итог попало скважин: {len(result['wells_used'])}")
    if len(dropped_summary):
        print("  Отброшено (по причинам):")
        display(dropped_summary)
    else:
        print("  Отброшенных скважин нет.")
    return result


# ----------------------------
# Пример использования
# ----------------------------
if 'df' in globals():
    print("Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...")
    cfg = PreprocConfig()
    out = preprocess_profiles(df, cfg)
    # Покажем заголовок long-таблицы как предварительный просмотр
    head_preview = out["panel_long"].head(12).copy()
    try:
        # Если доступна вспомогательная функция отображения таблиц — используем её
        import caas_jupyter_tools
        caas_jupyter_tools.display_dataframe_to_user("Предпросмотр panel_long", head_preview)
    except Exception:
        # Иначе просто печатаем
        print(head_preview)
else:
    print("Готово. Функции предобработки загружены. Для запуска вызовите:")
    print("cfg = PreprocConfig(T=36, min_profile_months=12)")
    print("out = preprocess_profiles(df, cfg)")
    print("panel_long, X = out['panel_long'], out['X']")


Обнаружен df в окружении. Запускаю предобработку с дефолтным конфигом...
Preprocess complete.
  В итог попало скважин: 2142
  Отброшено (по причинам):


,reason,count
0,no_start_detected,1930
1,too_short(<12),51


             well_name       date  oil_sum  water_sum  gas_sum  days_sum  \
0   25-003-05007-00-00 1986-01-01    252.0     2552.0      0.0      31.0   
1   25-003-05007-00-00 1986-02-01    239.0     2295.0      0.0      28.0   
2   25-003-05007-00-00 1986-03-01    253.0     2697.0      0.0      31.0   
3   25-003-05007-00-00 1986-04-01    249.0     1883.0      0.0      30.0   
4   25-003-05007-00-00 1986-05-01    278.0     2631.0      0.0      31.0   
5   25-003-05007-00-00 1986-06-01    244.0     2086.0      0.0      30.0   
6   25-003-05007-00-00 1986-07-01    271.0     2322.0      0.0      31.0   
7   25-003-05007-00-00 1986-09-01    195.0     1625.0      0.0       0.0   
8   25-003-05007-00-00 1986-10-01    224.0     1867.0      0.0       0.0   
9   25-003-05007-00-00 1986-11-01    209.0     1742.0      0.0       0.0   
10  25-003-05007-00-00 1986-12-01    224.0     1867.0      0.0       0.0   
11  25-003-05007-00-00 1987-01-01    221.0     2066.0      0.0       0.0   

    valid_d

In [8]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Шаг 2 (признаки) и Шаг 3 (Manifold)
-----------------------------------------------------------------------
Зависимости (pip): numpy, pandas, scikit-learn, umap-learn, fastdtw, tqdm (опц.)

Ожидается, что из Шага 1 у вас уже есть объект `out` с ключами:
  - out["panel_long"]: long-таблица по скважинам (после предобработки)
  - out["X"]: np.ndarray формы [n_wells, T, C] по каналам cfg.tensor_channels
  - out["wells_used"]: список имён скважин в том же порядке, что и ось 0 в X
  - out["tensor_channels"]: список имён каналов в X (например: ["r_oil_norm","wc","gor","dr_oil_norm"]) 

Этот файл добавляет:
  1) Шаг 2: компактные дескрипторы профилей (side features)
  2) Шаг 3: построение низкоразмерной "карты поведения" (UMAP) двумя способами:
       a) быстрый базовый (евклид на выровненных рядах)
       b) уточнённый с FastDTW, без полной O(N^2) — через доуточнение k ближайших пар

Примечание: библиотеку dtaidistance *не* используем. Для DTW берём fastdtw.
"""
from __future__ import annotations

import math
import warnings
from dataclasses import dataclass
from typing import Iterable, List, Optional, Sequence, Tuple, Dict

import numpy as np
import pandas as pd

# sklearn
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

# UMAP
try:
    import umap
except Exception as e:
    raise RuntimeError("Требуется пакет 'umap-learn'. Установите: pip install umap-learn")

# fastdtw
try:
    from fastdtw import fastdtw
except Exception:
    fastdtw = None
    warnings.warn("fastdtw не найден. DTW-вариант будет недоступен, останется базовый евклид.")

# tqdm (опционально)
try:
    from tqdm import tqdm
    _TQDM = True
except Exception:
    _TQDM = False


# ======================================================
# -------------------- ШАГ 2: ФИЧИ ---------------------
# ======================================================

def _nanpolyfit(y: np.ndarray, x: Optional[np.ndarray] = None, deg: int = 1) -> Tuple[float, ...]:
    """Лин. регрессия по ряду с NaN. Возвращает коэффициенты (последний — свободный член).
    deg=1 => (slope, intercept).
    """
    y = np.asarray(y, float)
    if x is None:
        x = np.arange(len(y), dtype=float)
    mask = np.isfinite(y) & np.isfinite(x)
    if mask.sum() < deg + 1:
        return tuple([np.nan] * (deg + 1))
    coeffs = np.polyfit(x[mask], y[mask], deg=deg)
    return tuple(coeffs)


def _first_index_where(y: np.ndarray, cond) -> Optional[int]:
    idx = np.nonzero(cond(y))[0]
    return int(idx[0]) if idx.size else None


def _rolling_stat(y: np.ndarray, win: int, fn) -> float:
    """Одно число: fn по всем скользящим окнам длины win, агрегация = среднее.
    Например, fn = np.std или пользовательская функция.
    """
    y = np.asarray(y, float)
    if win <= 0 or len(y) < win:
        return np.nan
    vals = []
    for i in range(len(y) - win + 1):
        seg = y[i:i+win]
        if np.isfinite(seg).sum() < max(2, win // 2):
            continue
        vals.append(fn(seg[np.isfinite(seg)]))
    return float(np.mean(vals)) if len(vals) else np.nan


def _slope_over_window(y: np.ndarray, win: int) -> float:
    if win <= 1 or len(y) < win:
        return np.nan
    slopes = []
    for i in range(len(y) - win + 1):
        seg = y[i:i+win]
        if np.isfinite(seg).sum() < max(2, win // 2):
            continue
        a, b = _nanpolyfit(seg, np.arange(win), deg=1)
        slopes.append(a)
    return float(np.mean(slopes)) if slopes else np.nan


def compute_side_features(panel_long: pd.DataFrame, T: int = 36) -> pd.DataFrame:
    """Строит компактные дескрипторы профиля по каждой скважине на горизонте [0..T).

    Требуемые колонки в panel_long: [well_name, t, r_oil_s, r_oil_norm, wc, gor, dr_oil_norm]
    Возвращает DataFrame: по одной строке на скважину.
    """
    need = {"well_name", "t", "r_oil_s", "r_oil_norm", "wc", "gor", "dr_oil_norm"}
    missing = need - set(panel_long.columns)
    if missing:
        raise ValueError(f"compute_side_features: не хватает колонок: {missing}")

    feats: List[pd.DataFrame] = []
    for w, g in panel_long.groupby("well_name", sort=False):
        g = g.copy()
        g = g[g["t"].between(0, T-1)]
        g = g.sort_values("t")
        # Векторы (длина может быть < T)
        oil = g["r_oil_s"].to_numpy(float)
        oil_norm = g["r_oil_norm"].to_numpy(float)
        wc = g["wc"].to_numpy(float)
        gor = g["gor"].to_numpy(float)
        d_oil = g["dr_oil_norm"].to_numpy(float)

        # Базовые величины
        peak = np.nanmax(oil) if oil.size else np.nan
        t_peak = int(np.nanargmax(oil)) if (oil.size and np.isfinite(peak)) else np.nan

        # Время до 50% и 20% от пика (decline speed)
        half = 0.5 * peak if np.isfinite(peak) else np.nan
        fifth = 0.2 * peak if np.isfinite(peak) else np.nan
        t_half = _first_index_where(oil, lambda y: np.isfinite(half) and (y <= half))
        t_20 = _first_index_where(oil, lambda y: np.isfinite(fifth) and (y <= fifth))

        # Усреднения по ранним/средним окнам
        m6 = int(min(6, len(oil)))
        m12 = int(min(12, len(oil)))
        early_mean6 = float(np.nanmean(oil_norm[:m6])) if m6 else np.nan
        early_mean12 = float(np.nanmean(oil_norm[:m12])) if m12 else np.nan
        early_trend6, _ = _nanpolyfit(oil_norm[:m6], np.arange(m6), 1) if m6 >= 3 else (np.nan, np.nan)
        mid_trend12 = _slope_over_window(oil_norm, 12)

        # Всплески/скачкообразность
        vol_doil_w6 = _rolling_stat(d_oil, win=6, fn=np.std)
        vol_doil_w12 = _rolling_stat(d_oil, win=12, fn=np.std)

        # Водоотдача и газовый фактор — уровни и тренды
        wc_mean12 = float(np.nanmean(wc[:m12])) if m12 else np.nan
        wc_trend12 = _slope_over_window(wc, 12)
        gor_trend12 = _slope_over_window(gor, 12)

        # Плато: сколько месяцев oil_norm >= 0.9 в начале профиля
        plateau_len = 0
        for v in oil_norm:
            if not np.isfinite(v) or v < 0.9:
                break
            plateau_len += 1

        # Системные пропуски
        valid_ratio = float(np.isfinite(oil).sum() / max(1, T))

        row = pd.DataFrame({
            "well_name": [w],
            "peak_oil": [peak],
            "t_peak": [t_peak],
            "t_half": [t_half],
            "t_20": [t_20],
            "early_mean6": [early_mean6],
            "early_mean12": [early_mean12],
            "early_trend6": [early_trend6],
            "mid_trend12": [mid_trend12],
            "vol_doil_w6": [vol_doil_w6],
            "vol_doil_w12": [vol_doil_w12],
            "wc_mean12": [wc_mean12],
            "wc_trend12": [wc_trend12],
            "gor_trend12": [gor_trend12],
            "plateau_len": [plateau_len],
            "valid_ratio": [valid_ratio],
        })
        feats.append(row)

    feats_df = pd.concat(feats, ignore_index=True) if feats else pd.DataFrame()
    return feats_df


def scale_features(feats_df: pd.DataFrame, exclude: Sequence[str] = ("well_name",)) -> Tuple[pd.DataFrame, RobustScaler]:
    if feats_df.empty:
        return feats_df, RobustScaler()
    cols = [c for c in feats_df.columns if c not in exclude]
    scaler = RobustScaler()
    X = scaler.fit_transform(feats_df[cols].astype(float).values)
    feats_scaled = feats_df.copy()
    for i, c in enumerate(cols):
        feats_scaled[c] = X[:, i]
    return feats_scaled, scaler


# ======================================================
# --------------- ШАГ 3: MANIFOLD (UMAP) ---------------
# ======================================================

@dataclass
class ManifoldConfig:
    # Какие каналы из тензора X брать для "формы"
    channels: Tuple[str, ...] = ("r_oil_norm", "wc")
    # DTW
    fastdtw_radius: int = 6
    k_refine: int = 40            # сколько ближайших по евклиду пар доуточнять DTW
    weights: Optional[Tuple[float, ...]] = (0.7, 0.3)  # веса каналов в многомерной метрике
    # UMAP
    n_neighbors: int = 30
    min_dist: float = 0.05
    n_components: int = 2
    random_state: int = 43


def _flatten_series_matrix(X: np.ndarray, channels_idx: Sequence[int]) -> np.ndarray:
    """Преобразует [N, T, C] -> [N, T*len(channels_idx)], NaN -> 0.
    Этот вектор используется ТОЛЬКО для быстрой евклидовой близости/UMAP-базы.
    """
    x = X[:, :, channels_idx]  # [N,T,C']
    N, T, C = x.shape
    x = np.transpose(x, (0, 2, 1)).reshape(N, C * T)
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    return x


def embed_umap_euclid(X: np.ndarray, tensor_channels: Sequence[str], channels: Sequence[str],
                      n_neighbors: int = 30, min_dist: float = 0.05, n_components: int = 2,
                      random_state: int = 42) -> Tuple[np.ndarray, umap.UMAP]:
    """Быстрый базовый manifold: UMAP по евклиду на выровненных рядах.
    Возвращает (Z, umap_model).
    """
    ch_to_idx = {c: i for i, c in enumerate(tensor_channels)}
    idx = [ch_to_idx[c] for c in channels if c in ch_to_idx]
    X2 = _flatten_series_matrix(X, idx)
    model = umap.UMAP(
        n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components,
        metric="euclidean", random_state=random_state
    )
    Z = model.fit_transform(X2)
    return Z, model


def _fastdtw_multichannel(A: np.ndarray, B: np.ndarray, weights: Optional[Sequence[float]] = None, radius: int = 6) -> float:
    """DTW для многоканальных рядов формы [T,C]. Использует L2 по каналам с весами.
    Отбрасывает позиции, где все каналы NaN. Заполняет оставшиеся NaN нулями.
    """
    if fastdtw is None:
        raise RuntimeError("fastdtw не установлен")
    A = np.asarray(A, float)
    B = np.asarray(B, float)
    if A.ndim != 2 or B.ndim != 2:
        raise ValueError("Ожидается [T,C] для обоих рядов")
    # маска валидных позиций (хотя бы один канал не NaN)
    maskA = np.any(np.isfinite(A), axis=1)
    maskB = np.any(np.isfinite(B), axis=1)
    A = A[maskA]
    B = B[maskB]
    if A.size == 0 or B.size == 0:
        return float("inf")
    # NaN -> 0 (после выкидывания полностью пустых кадров)
    A = np.nan_to_num(A, nan=0.0)
    B = np.nan_to_num(B, nan=0.0)
    # взвешивание каналов
    if weights is not None:
        w = np.asarray(weights, float).reshape(1, -1)
        if w.shape[1] != A.shape[1]:
            raise ValueError("Длина weights должна совпадать с числом каналов")
        A = A * w
        B = B * w
    # метрика между точками — евклид в C-мерном пространстве
    dist, _ = fastdtw(A, B, radius=radius, dist=2)
    return float(dist)


def embed_umap_fastdtw(
    X: np.ndarray,
    tensor_channels: Sequence[str],
    channels: Sequence[str],
    cfg: Optional[ManifoldConfig] = None,
    sample_size: Optional[int] = None,
    candidate_knn: Optional[int] = None,
) -> Tuple[np.ndarray, List[str], np.ndarray, Dict[str, object]]:
    """UMAP с уточнением FastDTW для ближайших пар.

    Алгоритм:
      1) Строим евклидову матрицу расстояний E в пространстве выровненных рядов (быстро).
      2) Для каждой точки берём k_refine ближайших по E и пересчитываем расстояния DTW (fastdtw).
      3) Получаем полную матрицу D: для уточнённых пар D=DTW, иначе D=E (масштаб сопоставим, но разный — это ок для UMAP).
      4) UMAP(metric='precomputed') на D.

    Если sample_size задан, берём случайную подвыборку для ускорения.

    Возвращает: (Z, wells_sub, D, info)
    """
    if cfg is None:
        cfg = ManifoldConfig()

    rng = np.random.default_rng(cfg.random_state)
    N, T, Ctot = X.shape
    ch_to_idx = {c: i for i, c in enumerate(tensor_channels)}
    idx = [ch_to_idx[c] for c in channels if c in ch_to_idx]
    if len(idx) != len(channels):
        missing = [c for c in channels if c not in ch_to_idx]
        raise ValueError(f"В тензоре X нет каналов: {missing}")

    # Подвыборка
    all_idx = np.arange(N)
    if sample_size is not None and sample_size < N:
        sub_idx = rng.choice(all_idx, size=sample_size, replace=False)
    else:
        sub_idx = all_idx
    Xs = X[sub_idx][:, :, idx]  # [Ns, T, C']

    # База: евклид на флеттене
    Xflat = _flatten_series_matrix(Xs, list(range(len(idx))))  # [Ns, T*C']
    E = pairwise_distances(Xflat, metric="euclidean")  # [Ns,Ns]

    Ns = Xs.shape[0]
    D = E.copy()

    if fastdtw is None:
        warnings.warn("fastdtw недоступен — возвращаю UMAP по евклиду (без уточнения DTW)")
    else:
        # Список пар для уточнения — top-k по E для каждой точки
        k = cfg.k_refine if candidate_knn is None else candidate_knn
        nn = NearestNeighbors(n_neighbors=min(k+1, Ns), metric="euclidean")
        nn.fit(Xflat)
        dists, inds = nn.kneighbors(Xflat, return_distance=True)
        # inds[:,0] — сама точка; начнём с 1..
        pairs = set()
        for i in range(Ns):
            for j in inds[i, 1:]:
                a, b = (int(i), int(j)) if i < j else (int(j), int(i))
                if a != b:
                    pairs.add((a, b))
        pairs = list(pairs)

        it = pairs
        if _TQDM:
            it = tqdm(pairs, desc=f"Recompute DTW for {len(pairs)} pairs (radius={cfg.fastdtw_radius})")

        for (i, j) in it:
            d = _fastdtw_multichannel(Xs[i], Xs[j], weights=cfg.weights, radius=cfg.fastdtw_radius)
            if not math.isfinite(d):
                continue
            D[i, j] = d
            D[j, i] = d

    # UMAP на предвычисленной матрице
    model = umap.UMAP(
        n_neighbors=cfg.n_neighbors,
        min_dist=cfg.min_dist,
        n_components=cfg.n_components,
        metric="precomputed",
        random_state=cfg.random_state,
    )
    Z = model.fit_transform(D)

    info = {
        "Ns": int(Ns),
        "channels": list(channels),
        "k_refine": int(cfg.k_refine),
        "fastdtw_radius": int(cfg.fastdtw_radius),
        "umap_params": {
            "n_neighbors": cfg.n_neighbors,
            "min_dist": cfg.min_dist,
            "n_components": cfg.n_components,
            "random_state": cfg.random_state,
        }
    }
    return Z, sub_idx.tolist(), D, info


# ======================================================
# ======================================================

if 'out' in globals():
    panel_long = out["panel_long"]
    X = out["X"]
    wells = out["wells_used"]
    tensor_channels = out["tensor_channels"]
    T = out["config"]["T"] if "config" in out else X.shape[1]

    # --- Шаг 2: компактные фичи
    feats = compute_side_features(panel_long, T=T)
    feats_scaled, scaler = scale_features(feats)
    print("Side features shape:", feats_scaled.shape)

    # --- Шаг 3a: быстрый UMAP по евклиду на рядах
    Z_euclid, umap_e = embed_umap_euclid(
        X, tensor_channels=tensor_channels, channels=["r_oil_norm", "wc"],
        n_neighbors=30, min_dist=0.05, n_components=2, random_state=42
    )

    # --- Шаг 3b: UMAP с FastDTW-уточнением
    cfg_m = ManifoldConfig(channels=("r_oil_norm", "wc"), fastdtw_radius=6, k_refine=40,
                           weights=(0.7, 0.3), n_neighbors=30, min_dist=0.05,
                           n_components=2, random_state=42)
    Z_dtw, sub_idx, D, info = embed_umap_fastdtw(
        X, tensor_channels=tensor_channels, channels=cfg_m.channels,
        cfg=cfg_m, sample_size=500  # можно None для полного набора
    )
    print("DTW-UMAP:", Z_dtw.shape, "subset size:", len(sub_idx))

    # Пример: собрать DataFrame с координатами
    df_map = pd.DataFrame({
        "well_name": np.array(wells)[sub_idx],
        "x": Z_dtw[:,0],
        "y": Z_dtw[:,1],
    })
    display(df_map.head())
else:
    print("Для примера использования требуется объект 'out' из Шага 1.")


Side features shape: (2142, 16)


Recompute DTW for 64583 pairs (radius=6): 100%|██████████| 64583/64583 [12:43<00:00, 84.55it/s] 


DTW-UMAP: (2142, 2) subset size: 2142


,well_name,x,y
0,25-003-05007-00-00,7.190036,9.426781
1,25-003-05009-00-00,7.079275,10.082273
2,25-003-05057-00-00,7.511279,10.024635
3,25-003-05067-00-00,6.941536,9.197872
4,25-003-05068-00-00,7.686371,10.157837


In [9]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Шаг 4: Сегментация и аномалии
------------------------------------------------------------------
Зависимости (pip): numpy, pandas, scikit-learn, hdbscan, (опц.) tslearn

Входы из предыдущих шагов:
  - out: результат preprocess_profiles (Шаг 1)
      * out["panel_long"] — long-таблица после выравнивания t=0
      * out["X"] — тензор [N, T, C]
      * out["wells_used"] — список имён в порядке X
      * out["tensor_channels"] — имена каналов в X
      * out["config"]["T"] — горизонт T
  - Z, sub_idx: embedding и индексы скважин, вошедшие в него (из Шага 3)

Функционал:
  1) Кластеризация HDBSCAN на manifold (поддержка шума/аномалий)
  2) Альтернатива: GMM по BIC
  3) Аномалии: LOF + расстояние до медоида кластера
  4) Прототипы кластеров (медианные профили и, опционально, soft-DTW/DTW барицентры)
  5) Метрики качества: Silhouette (без шума), DBCV (для HDBSCAN)
  6) Удобные хелперы для сборки отчёта

Примечание: dtaidistance НЕ используется. Для soft-DTW/DBA — опционально tslearn.
"""
from __future__ import annotations

import math
import warnings
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd

from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.mixture import GaussianMixture

# --- HDBSCAN ---
try:
    import hdbscan
except Exception as e:
    raise RuntimeError("Требуется пакет 'hdbscan'. Установите: pip install hdbscan")

# --- tslearn (опционально для барицентров) ---
try:
    from tslearn.barycenters import softdtw_barycenter, dtw_barycenter_averaging
    _TSLEARN_OK = True
except Exception:
    _TSLEARN_OK = False
    warnings.warn("tslearn не найден: прототипы будут как медианные кривые (без барицентров)")


# ======================================================
# ---------------- КЛАСТЕРИЗАЦИЯ -----------------------
# ======================================================

@dataclass
class ClusterConfig:
    # HDBSCAN
    min_cluster_size: int = 50
    min_samples: int = 12
    cluster_selection_epsilon: float = 0.0
    allow_single_cluster: bool = False
    # LOF
    lof_neighbors: int = 30


def cluster_hdbscan(
    Z: np.ndarray,
    wells_sub: Sequence[str],
    cfg: Optional[ClusterConfig] = None,
    metric: str = "euclidean",
) -> Dict[str, object]:
    """Кластеризация HDBSCAN на низкоразмерных координатах Z.

    Возвращает dict:
      labels: np.array [Ns]
      probabilities: np.array [Ns]
      clusterer: HDBSCAN объект
      df_map: DataFrame(well_name, x, y, cluster, prob)
      silhouette: float | np.nan (по точкам с cluster>=0)
      dbcv: float | np.nan
    """
    if cfg is None:
        cfg = ClusterConfig()

    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=cfg.min_cluster_size,
        min_samples=cfg.min_samples,
        metric=metric,
        cluster_selection_epsilon=cfg.cluster_selection_epsilon,
        allow_single_cluster=cfg.allow_single_cluster,
        prediction_data=True,
    )
    labels = clusterer.fit_predict(Z)
    probs = getattr(clusterer, "probabilities_", np.ones(len(labels)))

    # Карта
    df_map = pd.DataFrame({
        "well_name": wells_sub,
        "x": Z[:, 0],
        "y": Z[:, 1],
        "cluster": labels,
        "prob": probs,
    })

    # Метрики
    mask = labels >= 0
    if mask.sum() > 1 and len(np.unique(labels[mask])) > 1:
        try:
            sil = float(silhouette_score(Z[mask], labels[mask], metric="euclidean"))
        except Exception:
            sil = float("nan")
    else:
        sil = float("nan")

    try:
        dbcv = float(hdbscan.validity.validity_index(Z, labels, metric="euclidean"))
    except Exception:
        dbcv = float("nan")

    return {
        "labels": labels,
        "probabilities": probs,
        "clusterer": clusterer,
        "df_map": df_map,
        "silhouette": sil,
        "dbcv": dbcv,
    }


def cluster_gmm_bic(Z: np.ndarray, wells_sub: Sequence[str], n_range: Sequence[int] = range(2, 11)) -> Dict[str, object]:
    """Альтернативная кластеризация GMM с выбором числа компонент по BIC.
    Возвращает dict аналогичный HDBSCAN (labels, df_map, silhouette, model, best_k, bic_table).
    """
    best_bic = np.inf
    best = None
    rows = []
    for k in n_range:
        gm = GaussianMixture(n_components=k, covariance_type="full", random_state=42)
        gm.fit(Z)
        bic = gm.bic(Z)
        rows.append({"k": k, "bic": bic})
        if bic < best_bic:
            best_bic = bic
            best = gm
    bic_table = pd.DataFrame(rows)
    labels = best.predict(Z)
    probs = best.predict_proba(Z).max(axis=1)
    df_map = pd.DataFrame({"well_name": wells_sub, "x": Z[:,0], "y": Z[:,1], "cluster": labels, "prob": probs})
    sil = float("nan")
    mask = labels >= 0
    if len(np.unique(labels[mask])) > 1:
        try:
            sil = float(silhouette_score(Z[mask], labels[mask], metric="euclidean"))
        except Exception:
            pass
    return {
        "labels": labels,
        "probabilities": probs,
        "model": best,
        "best_k": int(np.unique(labels).size),
        "df_map": df_map,
        "silhouette": sil,
        "bic_table": bic_table,
    }


# ======================================================
# ------------------- АНОМАЛИИ -------------------------
# ======================================================

def lof_anomaly_scores(Z: np.ndarray, n_neighbors: int = 30) -> np.ndarray:
    """Возвращает аномальность в [0..1], где 1 ~ сильная аномалия.
    На основе LocalOutlierFactor (scikit-learn), преобразуя negative_outlier_factor_.
    """
    lof = LocalOutlierFactor(n_neighbors=min(n_neighbors, len(Z) - 1), novelty=False)
    lof.fit(Z)
    # Чем более отрицателен negative_outlier_factor_, тем более аномальна точка.
    s = -lof.negative_outlier_factor_
    s = (s - np.min(s)) / (np.ptp(s) + 1e-12)
    return s


def distance_to_medoid(Z: np.ndarray, labels: np.ndarray) -> np.ndarray:
    """Расстояние точки до медоида своего кластера в координатах embedding.
    Для шума (-1) — расстояние до ближайшего кластера.
    """
    uniq = [c for c in np.unique(labels) if c >= 0]
    if not len(uniq):
        return np.full(len(labels), np.nan)
    medoids = {}
    for c in uniq:
        idx = np.where(labels == c)[0]
        if idx.size == 0:
            continue
        D = pairwise_distances(Z[idx], metric="euclidean")
        m = idx[np.argmin(D.sum(axis=0))]
        medoids[c] = Z[m]
    out = np.zeros(len(labels))
    for i, lab in enumerate(labels):
        if lab >= 0:
            mu = medoids.get(lab)
            out[i] = float(np.linalg.norm(Z[i] - mu))
        else:
            # для шума — ближайший медоид
            dmin = np.min([np.linalg.norm(Z[i] - mu) for mu in medoids.values()])
            out[i] = float(dmin)
    # нормализация [0..1]
    out = (out - np.min(out)) / (np.ptp(out) + 1e-12)
    return out


# ======================================================
# --------------- ПРОТОТИПЫ КЛАСТЕРОВ ------------------
# ======================================================

def _collect_matrix(panel_long: pd.DataFrame, wells: Sequence[str], channel: str, T: int) -> np.ndarray:
    """Собирает матрицу [n_series, T] по указанному каналу с NaN.
    """
    rows = []
    for w in wells:
        g = panel_long.loc[panel_long["well_name"] == w, ["t", channel]].sort_values("t")
        v = np.full(T, np.nan, float)
        t = g["t"].to_numpy(int)
        vals = g[channel].to_numpy(float)
        t = t[(t >= 0) & (t < T)]
        vals = vals[: len(t)]
        v[t[: len(vals)]] = vals
        rows.append(v)
    return np.vstack(rows) if rows else np.empty((0, T))


def _barycenter_or_median(M: np.ndarray, method: str = "auto", gamma: float = 1.0, max_iter: int = 50) -> np.ndarray:
    """Возвращает барицентр (soft-DTW/DBA) или медиану по времени, если tslearn недоступен.
    NaN заполняются нулями перед барицентром (так как tslearn не поддерживает NaN).
    """
    if M.size == 0:
        return M
    if method == "auto":
        method = "softdtw" if _TSLEARN_OK else "median"
    if method in ("softdtw", "dba") and not _TSLEARN_OK:
        method = "median"

    if method == "median":
        return np.nanmedian(M, axis=0)
    M_filled = np.nan_to_num(M, nan=0.0)
    if method == "softdtw":
        try:
            return softdtw_barycenter(M_filled, gamma=gamma, max_iter=max_iter)
        except Exception:
            return np.nanmedian(M, axis=0)
    if method == "dba":
        try:
            return dtw_barycenter_averaging(M_filled, max_iter=max_iter)
        except Exception:
            return np.nanmedian(M, axis=0)
    # fallback
    return np.nanmedian(M, axis=0)


def build_cluster_prototypes(
    panel_long: pd.DataFrame,
    df_map: pd.DataFrame,
    channels: Sequence[str] = ("r_oil_s", "wc", "gor", "r_oil_norm"),
    T: int = 36,
    method: str = "auto",   # 'auto'|'softdtw'|'dba'|'median'
    gamma: float = 1.0,
    max_iter: int = 50,
) -> Dict[int, Dict[str, np.ndarray]]:
    """Строит прототипы (по-канально) для каждого кластера (cluster>=0).
    Возвращает: {cluster_id: {channel: proto_vec_length_T}}
    Также можно использовать для отрисовки IQR/квантили — верните сырой M при необходимости.
    """
    # Скважины по кластерам
    cl2wells: Dict[int, List[str]] = {}
    for row in df_map.itertuples(index=False):
        if row.cluster >= 0:
            cl2wells.setdefault(int(row.cluster), []).append(str(row.well_name))

    res: Dict[int, Dict[str, np.ndarray]] = {}
    for cl, wells in cl2wells.items():
        res[cl] = {}
        for ch in channels:
            M = _collect_matrix(panel_long, wells, ch, T)
            proto = _barycenter_or_median(M, method=method, gamma=gamma, max_iter=max_iter)
            res[cl][ch] = proto
    return res


# ======================================================
# -------------- УТИЛИТЫ ДЛЯ ОТЧЁТА --------------------
# ======================================================

def summarize_clusters(df_map: pd.DataFrame) -> pd.DataFrame:
    """Возвращает сводную таблицу: размер кластера, медиана prob, доля шума."""
    total = len(df_map)
    noise_share = (df_map["cluster"].eq(-1)).mean() if total else np.nan
    rows = []
    for cl, g in df_map.groupby("cluster"):
        rows.append({
            "cluster": int(cl),
            "size": int(len(g)),
            "share": float(len(g) / total) if total else np.nan,
            "prob_median": float(np.median(g["prob"].values)) if len(g) else np.nan,
        })
    out = pd.DataFrame(rows).sort_values(["cluster"]).reset_index(drop=True)
    out.attrs["noise_share"] = float(noise_share)
    return out


def assign_anomaly_scores(df_map: pd.DataFrame, Z: np.ndarray, labels: np.ndarray, lof_k: int = 30) -> pd.DataFrame:
    """Добавляет к df_map столбцы: lof_score [0..1], dist_medoid [0..1], anomaly_score (среднее двух)."""
    lof = lof_anomaly_scores(Z, n_neighbors=lof_k)
    dmed = distance_to_medoid(Z, labels)
    anom = 0.5 * (lof + dmed)
    out = df_map.copy()
    out["lof_score"] = lof
    out["dist_medoid"] = dmed
    out["anomaly_score"] = anom
    return out


# ======================================================
# -------------- ПРИМЕР ПОСЛЕДОВАТЕЛЬНОСТИ -------------
# ======================================================

if 'out' in globals() and 'Z_dtw' in globals() and 'sub_idx' in globals():
    panel_long = out["panel_long"]
    wells_all = out["wells_used"]
    T = out["config"].get("T", 36)
    wells_sub = np.array(wells_all)[sub_idx].tolist()

    # 1) HDBSCAN
    cfg = ClusterConfig(min_cluster_size=50, min_samples=12)
    res = cluster_hdbscan(Z_dtw, wells_sub, cfg)
    df_map = res["df_map"]
    print("Silhouette:", res["silhouette"], " DBCV:", res["dbcv"])
    print(summarize_clusters(df_map))

    # 2) Аномалии
    df_map = assign_anomaly_scores(df_map, Z_dtw, res["labels"], lof_k=30)
    # пример отбора аномалий
    anomalies = df_map.sort_values("anomaly_score", ascending=False).head(30)

    # 3) Прототипы (медианные или soft-DTW / DBA при наличии tslearn)
    protos = build_cluster_prototypes(panel_long, df_map, channels=("r_oil_s","wc","gor","r_oil_norm"), T=T, method="auto")

    # Теперь df_map можно соединить с геолого-технологическими признаками и строить объяснения.
else:
    print("Для примера нужен 'out' (Шаг 1) и 'Z_dtw, sub_idx' (Шаг 3).")


Silhouette: 0.6330978870391846  DBCV: nan
   cluster  size     share  prob_median
0       -1   662  0.309057     0.000000
1        0    97  0.045285     1.000000
2        1    54  0.025210     1.000000
3        2    85  0.039683     1.000000
4        3    96  0.044818     1.000000
5        4   186  0.086835     0.876985
6        5   120  0.056022     0.942701
7        6   338  0.157796     0.815296
8        7   169  0.078898     0.903724
9        8   335  0.156396     1.000000


In [10]:
# -*- coding: utf-8 -*-
"""
Production Behavior Manifold (PBM) — Шаг 5: Визуализации и Отчёты
------------------------------------------------------------------
Зависимости (pip): numpy, pandas, matplotlib, (опц.) jinja2

Правила построения графиков (важно для окружения чат-ноутбука):
  - использовать matplotlib (не seaborn)
  - один график на фигуру (без субплотов)
  - не задавать явные цвета (использовать дефолтные)

Входы из предыдущих шагов:
  - out: результат preprocess_profiles (Шаг 1)
  - Z, sub_idx: embedding и индексы скважин, вошедшие в него (Шаг 3)
  - res: результат cluster_hdbscan или cluster_gmm_bic (Шаг 4) с полем df_map
  - protos: прототипы (build_cluster_prototypes)

Основные функции:
  * save_pbm_map(...) — карта PBM с кластерами/аномалиями
  * save_cluster_prototype_plots(...) — профили прототипов с IQR-заштриховкой (если доступна сырая матрица)
  * save_cluster_distribution_plot(...) — диаграмма распределения размеров кластеров
  * export_csv_summaries(...) — CSV-выгрузки (карта, сводка по кластерам, топ аномалий)
  * build_html_report(...) — простой HTML-отчёт с картинками и таблицами

Пример использования внизу файла (закомментирован).
"""
from __future__ import annotations

import os
import io
import math
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # рендер без GUI
import matplotlib.pyplot as plt

# ------------------------- УТИЛИТЫ -------------------------

def _ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def _collect_matrix(panel_long: pd.DataFrame, wells: Sequence[str], channel: str, T: int) -> np.ndarray:
    rows = []
    for w in wells:
        g = panel_long.loc[panel_long["well_name"] == w, ["t", channel]].sort_values("t")
        v = np.full(T, np.nan, float)
        t = g["t"].to_numpy(int)
        vals = g[channel].to_numpy(float)
        t = t[(t >= 0) & (t < T)]
        vals = vals[: len(t)]
        v[t[: len(vals)]] = vals
        rows.append(v)
    return np.vstack(rows) if rows else np.empty((0, T))


# ----------------------- КАРТА PBM ------------------------

def save_pbm_map(
    Z: np.ndarray,
    df_map: pd.DataFrame,
    out_dir: str,
    title: str = "PBM map (UMAP)",
    annotate_medoids: bool = True,
    mark_anomalies: bool = True,
    anomaly_col: str = "anomaly_score",
    dpi: int = 160,
) -> str:
    """Сохраняет PNG с картой PBM (scatter), один график — одна фигура.
    Возвращает путь к PNG.
    """
    _ensure_dir(out_dir)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_title(title)
    # точки
    sc = ax.scatter(Z[:, 0], Z[:, 1], s=12)
    ax.set_xlabel("UMAP-1")
    ax.set_ylabel("UMAP-2")

    # Медоиды кластеров (на координатах Z)
    if annotate_medoids and "cluster" in df_map.columns:
        from sklearn.metrics import pairwise_distances
        for cl, g in df_map[df_map["cluster"] >= 0].groupby("cluster"):
            idx = g.index.to_numpy()
            D = pairwise_distances(Z[idx], metric="euclidean")
            m_local = int(idx[np.argmin(D.sum(axis=0))])
            ax.text(Z[m_local, 0], Z[m_local, 1], f"C{int(cl)}", fontsize=9)

    # Аномалии (обводка поверх, если есть колонка)
    if mark_anomalies and anomaly_col in df_map.columns:
        q = df_map[anomaly_col].quantile(0.95)
        mask = df_map[anomaly_col] >= q
        ax.scatter(Z[mask, 0], Z[mask, 1], s=32, facecolors='none', edgecolors='k', linewidths=0.8)

    path = os.path.join(out_dir, "pbm_map.png")
    fig.tight_layout()
    fig.savefig(path, dpi=dpi)
    plt.close(fig)
    return path


# --------------- РАСПРЕДЕЛЕНИЕ КЛАСТЕРОВ ----------------

def save_cluster_distribution_plot(df_map: pd.DataFrame, out_dir: str, dpi: int = 160) -> str:
    _ensure_dir(out_dir)
    sizes = df_map[df_map["cluster"] >= 0]["cluster"].value_counts().sort_index()
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.set_title("Cluster sizes")
    ax.bar(sizes.index.astype(str), sizes.values)
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Count")
    path = os.path.join(out_dir, "cluster_sizes.png")
    fig.tight_layout()
    fig.savefig(path, dpi=dpi)
    plt.close(fig)
    return path


# ------------------ ПРОФИЛИ ПРОТОТИПОВ -------------------

def save_cluster_prototype_plots(
    panel_long: pd.DataFrame,
    df_map: pd.DataFrame,
    protos: Dict[int, Dict[str, np.ndarray]],
    channels: Sequence[str],
    T: int,
    out_dir: str,
    dpi: int = 160,
) -> List[str]:
    """Для каждого кластера и канала рисует один график с прототипом и IQR-зоной.
    Возвращает список путей к PNG.
    """
    _ensure_dir(out_dir)
    paths: List[str] = []
    # подготовка списков скважин по кластеру
    cl2wells: Dict[int, List[str]] = {}
    for row in df_map.itertuples(index=False):
        if row.cluster >= 0:
            cl2wells.setdefault(int(row.cluster), []).append(str(row.well_name))

    for cl, ch_dict in protos.items():
        wells = cl2wells.get(cl, [])
        for ch in channels:
            proto = ch_dict.get(ch)
            if proto is None:
                continue
            # Матрица ряда для IQR
            M = _collect_matrix(panel_long, wells, ch, T)
            p25 = np.nanpercentile(M, 25, axis=0) if M.size else None
            p75 = np.nanpercentile(M, 75, axis=0) if M.size else None
            x = np.arange(len(proto))
            fig, ax = plt.subplots(figsize=(8, 4))
            ax.set_title(f"Cluster {cl} — {ch}")
            ax.plot(x, proto)
            if M.size:
                ax.fill_between(x, p25, p75, alpha=0.25)
            ax.set_xlabel("t (months since start)")
            ax.set_ylabel(ch)
            path = os.path.join(out_dir, f"cluster_{cl}_{ch}.png")
            fig.tight_layout()
            fig.savefig(path, dpi=dpi)
            plt.close(fig)
            paths.append(path)
    return paths


# ----------------------- CSV-ЭКСПОРТ ---------------------

def export_csv_summaries(
    df_map: pd.DataFrame,
    summary: pd.DataFrame,
    out_dir: str,
    top_anoms: int = 50,
) -> Dict[str, str]:
    _ensure_dir(out_dir)
    paths = {}
    p_map = os.path.join(out_dir, "pbm_map_points.csv")
    df_map.to_csv(p_map, index=False)
    paths["map_csv"] = p_map

    p_sum = os.path.join(out_dir, "cluster_summary.csv")
    summary.to_csv(p_sum, index=False)
    paths["summary_csv"] = p_sum

    if "anomaly_score" in df_map.columns:
        p_an = os.path.join(out_dir, "top_anomalies.csv")
        df_map.sort_values("anomaly_score", ascending=False).head(top_anoms).to_csv(p_an, index=False)
        paths["anomalies_csv"] = p_an
    return paths


# --------------------- HTML-ОТЧЁТ -----------------------

def build_html_report(
    out_dir: str,
    map_png: str,
    sizes_png: str,
    proto_pngs: Sequence[str],
    df_map: pd.DataFrame,
    summary: pd.DataFrame,
    title: str = "PBM Report",
    filename: str = "PBM_report.html",
) -> str:
    """Простой статический HTML-отчёт без внешних шаблонизаторов.
    Картинки и таблицы вставляются как <img> и <table>.
    """
    _ensure_dir(out_dir)
    def df_to_html(df: pd.DataFrame) -> str:
        # Минимальная чистка HTML
        return df.to_html(index=False, escape=True)

    html = [
        "<!DOCTYPE html>",
        "<html><head><meta charset='utf-8'><title>{}</title>".format(title),
        "<style>body{font-family:Arial, sans-serif; margin:24px;} h2{margin-top:28px;} img{max-width:100%; height:auto;} table{border-collapse:collapse;} td,th{border:1px solid #ddd; padding:6px;}</style>",
        "</head><body>",
        f"<h1>{title}</h1>",
        "<h2>PBM Map</h2>",
        f"<img src='{os.path.relpath(map_png, out_dir)}' alt='PBM map'>",
        "<h2>Cluster Sizes</h2>",
        f"<img src='{os.path.relpath(sizes_png, out_dir)}' alt='Cluster sizes'>",
        "<h2>Cluster Summary</h2>",
        df_to_html(summary),
        "<h2>Top Anomalies</h2>",
        df_to_html(df_map.sort_values('anomaly_score', ascending=False).head(50) if 'anomaly_score' in df_map.columns else df_map.head(50)),
        "<h2>Cluster Prototypes</h2>",
    ]
    # галерея прототипов
    for p in proto_pngs:
        html.append(f"<img src='{os.path.relpath(p, out_dir)}' alt='{os.path.basename(p)}'>")
    html.append("</body></html>")

    out_path = os.path.join(out_dir, filename)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(html))
    return out_path


# -------------------- ПРИМЕР ИСПОЛЬЗОВАНИЯ --------------------
# (раскомментируйте и выполните после Шагов 1–4)
#
# if 'out' in globals() and 'Z_dtw' in globals() and 'sub_idx' in globals() and 'res' in globals() and 'protos' in globals():
#     panel_long = out["panel_long"]
#     T = out["config"].get("T", 36)
#     df_map = res["df_map"]
#     out_dir = "./pbm_report_exports"  # или '/mnt/data/pbm_report_exports' в окружении чата
# 
#     # 1) Основные картинки
#     map_png = save_pbm_map(Z_dtw, df_map, out_dir)
#     sizes_png = save_cluster_distribution_plot(df_map, out_dir)
# 
#     # 2) Прототипы (oil_s, wc, gor, r_oil_norm)
#     channels = ("r_oil_s","wc","gor","r_oil_norm")
#     proto_pngs = save_cluster_prototype_plots(panel_long, df_map, protos, channels, T, out_dir)
# 
#     # 3) Таблицы CSV
#     summary = summarize_clusters(df_map)
#     csv_paths = export_csv_summaries(df_map, summary, out_dir, top_anoms=50)
# 
#     # 4) HTML отчёт
#     report_path = build_html_report(out_dir, map_png, sizes_png, proto_pngs, df_map, summary, title="PBM Report")
#     print("Готов отчёт:", report_path)
# else:
#     print("Для примера нужен 'out', 'Z_dtw','sub_idx','res','protos'.")


Готов отчёт: ./pbm_report_exports/PBM_report.html


In [11]:
panel_long = out["panel_long"]
T = out["config"]["T"]
df_map = res["df_map"]  # из HDBSCAN/GMM
out_dir = "./pbm_report_exports"

map_png   = save_pbm_map(Z_dtw, df_map, out_dir)              # карта PBM
sizes_png = save_cluster_distribution_plot(df_map, out_dir)   # размеры кластеров
proto_pngs = save_cluster_prototype_plots(
    panel_long, df_map, protos, channels=("r_oil_s","wc","gor","r_oil_norm"), T=T, out_dir=out_dir
)

summary = summarize_clusters(df_map)
csvs = export_csv_summaries(df_map, summary, out_dir, top_anoms=50)

report = build_html_report(out_dir, map_png, sizes_png, proto_pngs, df_map, summary, title="PBM Report")
print("Отчёт:", report)

Отчёт: ./pbm_report_exports/PBM_report.html



# Шаг 6. Прогноз профиля по префиксу (20 → 100)

**Цель:** прогнозировать месяцы 21–100 для каждой скважины, используя только первые 20 месяцев и без утечки информации.

Подходы в этом шаге:
1. **KNN-достройка по префиксу** с амплитудным выравниванием соседей (по МНК на префиксе).
2. **Мультивыходная ElasticNet-регрессия** на компактных признаках префикса.

Метрики: RMSE и sMAPE на окне 21–100.  
Отчёты и артефакты сохраняются в `./forecast_exports`.


In [16]:

"""
forecast_addon.py — Utilities for prefix→suffix forecasting on well production profiles.

Assumptions:
- You have a long-format DataFrame `panel_long` with columns: well_name, t (0..T-1), r_oil_s, wc, gor.
- The preprocessing aligned profiles so t=0 is production start (already done in your PBM Step 1).
- `out` is a dict from your pipeline with keys: "panel_long", "wells_used", "config" with {"T": int}.

This module provides:
- build_prefix_scaled_channel(): scale r_oil_s by prefix-quantile for leakage-free comparison
- make_matrices(): construct prefix matrix X_pref and full target matrix Y_full
- knn_forecast(): neighbor-based completion with per-neighbor amplitude alignment
- multioutput_forecast(): elastic-net multi-output baseline on compact prefix features
- evaluate_forecasts(): RMSE and sMAPE metrics
"""

import numpy as np
import pandas as pd
from typing import Tuple, Dict, Optional

def build_prefix_scaled_channel(panel_long: pd.DataFrame, wells: list, T:int, T_pref:int,
                                q:float=0.90, eps:float=1e-9, clip_max:float=3.0,
                                rate_col:str="r_oil_s", out_col:str="r_oil_pref_norm") -> pd.DataFrame:
    """Create leakage-free, prefix-scaled oil-rate channel (normalized by prefix q-quantile)."""
    pl = panel_long.copy()
    pl = pl[pl["t"].between(0, T-1)].copy()
    # compute prefix scale per well
    scales = (
        pl[pl["t"] < T_pref]
        .groupby("well_name")[rate_col]
        .quantile(q)
        .rename("scale")
        .reset_index()
    )
    pl = pl.merge(scales, on="well_name", how="left")
    pl[out_col] = (pl[rate_col] / (pl["scale"].abs() + eps)).clip(lower=0, upper=clip_max)
    return pl.drop(columns=["scale"])

def make_matrices(panel_long: pd.DataFrame, wells: list, T:int, T_pref:int,
                  channel:str="r_oil_pref_norm", target_col:str="r_oil_s") -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Return X_pref [N, T_pref], Y_suffix [N, T-T_pref], Y_full [N, T] for the given channels."""
    idx = {w:i for i,w in enumerate(wells)}
    N = len(wells)
    X = np.full((N, T_pref), np.nan)
    Yfull = np.full((N, T), np.nan)
    for w, g in panel_long.groupby("well_name", sort=False):
        i = idx.get(w, None)
        if i is None: continue
        gg = g.sort_values("t")
        t = gg["t"].to_numpy()
        if len(t)==0: continue
        # fill prefix channel
        v = gg[channel].to_numpy()
        mask_pref = (t >= 0) & (t < T_pref)
        if mask_pref.any():
            X[i, t[mask_pref]] = v[mask_pref]
        # fill full target
        vy = gg[target_col].to_numpy()
        mask_full = (t >= 0) & (t < T)
        if mask_full.any():
            Yfull[i, t[mask_full]] = vy[mask_full]
    Ysuffix = Yfull[:, T_pref:T]
    return X, Ysuffix, Yfull

def _align_scale(y_ref_pref: np.ndarray, y_nei_pref: np.ndarray, eps:float=1e-9) -> float:
    """Least-squares scale factor to match neighbor prefix to reference prefix."""
    num = np.nansum(y_ref_pref * y_nei_pref)
    den = np.nansum(y_nei_pref ** 2) + eps
    s = num / den
    if not np.isfinite(s):
        s = 1.0
    return float(s)

def knn_forecast(X_pref: np.ndarray, Y_full: np.ndarray, T_pref:int, K:int=15) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
    """KNN completion with per-neighbor amplitude alignment on prefix. Returns pred_suffix [N, T-T_pref]."""
    from sklearn.neighbors import NearestNeighbors
    N, T_pref_ = X_pref.shape
    assert T_pref_ == T_pref
    # Use only wells with complete full horizon for training & evaluation
    mask_full = np.isfinite(Y_full).sum(axis=1) >= Y_full.shape[1]
    I = np.where(mask_full)[0]
    if len(I) < 3:
        raise ValueError("Not enough wells with full horizon for KNN.")
    nbrs = NearestNeighbors(n_neighbors=min(K+1, len(I)), metric="euclidean").fit(X_pref[I])
    dists, knn_idx = nbrs.kneighbors(X_pref[I])
    T_suffix = Y_full.shape[1] - T_pref
    pred = np.zeros((len(I), T_suffix))
    used = []
    for r, (row_knn, row_d) in enumerate(zip(knn_idx, dists)):
        # exclude self (distance ~ 0)
        neigh = [j for j,d in zip(row_knn, row_d) if d>0][:K]
        if not neigh:
            # fallback: take the single closest even if self
            neigh = [row_knn[0]]
        # gather neighbors (global indices)
        neigh_g = I[neigh]
        # align each neighbor by LS scale over prefix (on raw target y since it's in original units)
        y_ref_pref = Y_full[I[r], :T_pref]
        suffixes = []
        for g in neigh_g:
            y_nei_pref = Y_full[g, :T_pref]
            s = _align_scale(y_ref_pref, y_nei_pref)
            y_nei_suffix = s * Y_full[g, T_pref:]
            suffixes.append(y_nei_suffix)
        suffixes = np.vstack(suffixes)
        pred[r] = np.nanmedian(suffixes, axis=0)
        used.append(neigh_g)
    # Build a full-N predictions array filled with nan, then place known rows
    Ypred_fullN = np.full((N, T_suffix), np.nan)
    Ypred_fullN[I] = pred
    return Ypred_fullN, {"train_indices": I, "neighbors": used}

def _safe_import_build_side_features():
    try:
        # Inherit from the user's notebook if defined
        from build_side_features import build_side_features  # unlikely
        return build_side_features
    except Exception:
        return None

def _fallback_prefix_features(panel_long: pd.DataFrame, wells:list, T_pref:int) -> pd.DataFrame:
    """Compact features from prefix window: mean, std, slope, curvature, wc stats."""
    rows = []
    for w, g in panel_long[panel_long["t"].between(0, T_pref-1)].groupby("well_name", sort=False):
        gg = g.sort_values("t")
        t = gg["t"].to_numpy().astype(float)
        y = gg["r_oil_pref_norm"].to_numpy()
        wc = gg.get("wc", pd.Series([np.nan]*len(gg))).to_numpy()
        # basic stats
        mu, sd = np.nanmean(y), np.nanstd(y)
        # slope via linear fit
        if len(t) >= 2 and np.nanstd(t) > 0:
            A = np.vstack([t, np.ones_like(t)]).T
            m, b = np.linalg.lstsq(A, y, rcond=None)[0]
        else:
            m, b = 0.0, float(y[0]) if len(y) else 0.0
        # curvature (2nd diff)
        if len(y) >= 3:
            curv = np.nanmean(np.diff(y,2))
        else:
            curv = 0.0
        rows.append({
            "well_name": w,
            "y_mean": mu,
            "y_std": sd,
            "y_slope": m,
            "y_intercept": b,
            "y_curv": curv,
            "wc_mean": float(np.nanmean(wc)),
            "wc_std": float(np.nanstd(wc)),
        })
    return pd.DataFrame(rows)

def multioutput_forecast(panel_long: pd.DataFrame, wells:list, T:int, T_pref:int,
                         Y_full: np.ndarray, random_state:int=43) -> Tuple[np.ndarray, Dict]:
    """ElasticNetCV-based multioutput regression on compact prefix features."""
    from sklearn.linear_model import ElasticNetCV
    from sklearn.multioutput import MultiOutputRegressor
    # Build compact features
    feats = _fallback_prefix_features(panel_long, wells, T_pref)
    # align feature rows with wells order
    idx = {w:i for i,w in enumerate(wells)}
    X = np.full((len(wells), feats.shape[1]-1), np.nan)
    for _, r in feats.iterrows():
        i = idx[r["well_name"]]
        X[i] = r.drop(labels=["well_name"]).to_numpy(dtype=float)
    # training mask (full horizon)
    mask_full = np.isfinite(Y_full).sum(axis=1) >= T
    I = np.where(mask_full)[0]
    if len(I) < 3:
        raise ValueError("Not enough wells with full horizon for MultiOutput.")
    Y = Y_full[I, T_pref:T]
    model = MultiOutputRegressor(ElasticNetCV(l1_ratio=[0.1,0.5,0.9], cv=5, random_state=random_state))
    model.fit(X[I], Y)
    Ypred = np.full((len(wells), T-T_pref), np.nan)
    Ypred[I] = model.predict(X[I])
    return Ypred, {"train_indices": I, "model": model, "features": feats}

def evaluate_forecasts(Y_true, Y_pred):
    """RMSE и sMAPE по строкам, где и факт, и прогноз без NaN; без зависимости от 'squared'."""
    import numpy as np
    mask = np.isfinite(Y_pred).all(axis=1) & np.isfinite(Y_true).all(axis=1)
    n_eval = int(mask.sum())
    if n_eval == 0:
        return {"rmse": float("nan"), "smape": float("nan"), "n_eval": 0}
    diff = Y_pred[mask] - Y_true[mask]
    rmse = float(np.sqrt(np.mean(diff**2)))
    smape = float(np.nanmean(2*np.abs(Y_pred[mask]-Y_true[mask])/(np.abs(Y_pred[mask])+np.abs(Y_true[mask])+1e-9)))
    return {"rmse": rmse, "smape": smape, "n_eval": n_eval}

In [17]:
# === Шаг 6: Прогноз профиля по префиксу (20 → 100) ===
import os, numpy as np, pandas as pd, json
from datetime import datetime
import matplotlib.pyplot as plt

assert 'out' in globals(), "Требуется объект 'out' из Шага 1 (preprocess_profiles)."
panel_long = out["panel_long"].copy()
wells_used = out["wells_used"]
T = int(out["config"]["T"])
T_pref = 20  # можно вынести в конфиг

# 6.1. Построить префикс-нормированный канал без утечки
panel_long = build_prefix_scaled_channel(panel_long, wells_used, T=T, T_pref=T_pref,
                                         q=0.90, rate_col="r_oil_s", out_col="r_oil_pref_norm")

# 6.2. Матрицы X_pref, Y_suffix, Y_full
X_pref, Y_suffix_true, Y_full = make_matrices(panel_long, wells_used, T=T, T_pref=T_pref,
                                              channel="r_oil_pref_norm", target_col="r_oil_s")

# 6.3. KNN-достройка
Y_pred_knn, knn_info = knn_forecast(X_pref, Y_full, T_pref=T_pref, K=15)

# 6.4. Мультивыходная регрессия
Y_pred_lr, lr_info = multioutput_forecast(panel_long, wells_used, T=T, T_pref=T_pref, Y_full=Y_full, random_state=43)

# 6.5. Оценка качества
m_knn = evaluate_forecasts(Y_suffix_true, Y_pred_knn)
m_lr  = evaluate_forecasts(Y_suffix_true, Y_pred_lr)
print("KNN   → RMSE={rmse:.4f}, sMAPE={smape:.4f}, N={n_eval}".format(**m_knn))
print("ENet  → RMSE={rmse:.4f}, sMAPE={smape:.4f}, N={n_eval}".format(**m_lr))

# 6.6. Сохранение прогнозов и отчёта
out_dir = "./forecast_exports"
os.makedirs(out_dir, exist_ok=True)

np.save(os.path.join(out_dir, "Y_suffix_true.npy"), Y_suffix_true)
np.save(os.path.join(out_dir, "Y_pred_knn.npy"), Y_pred_knn)
np.save(os.path.join(out_dir, "Y_pred_enet.npy"), Y_pred_lr)

# Таблица метрик
metrics_df = pd.DataFrame([
    {"model": "knn", "rmse": m_knn["rmse"], "smape": m_knn["smape"], "n_eval": m_knn["n_eval"]},
    {"model": "elasticnet", "rmse": m_lr["rmse"], "smape": m_lr["smape"], "n_eval": m_lr["n_eval"]},
])
metrics_csv = os.path.join(out_dir, "metrics.csv")
metrics_df.to_csv(metrics_csv, index=False)

# Пер-скважинные прогнозы (с именами)
def save_predictions_csv(Y_pred: np.ndarray, wells: list, name:str):
    cols = [f"m{t}" for t in range(T_pref+1, T+1)]
    df = pd.DataFrame(Y_pred, columns=cols)
    df.insert(0, "well_name", wells)
    df.to_csv(os.path.join(out_dir, f"pred_{name}.csv"), index=False)

save_predictions_csv(Y_pred_knn, wells_used, "knn")
save_predictions_csv(Y_pred_lr,  wells_used, "elasticnet")

# 6.7. Примеры графиков "факт vs прогноз"
def plot_example(idx, title, ytrue, ypred):
    plt.figure()
    plt.plot(range(T_pref, T), ytrue[idx], label="true")
    plt.plot(range(T_pref, T), ypred[idx], label="pred")
    plt.title(title)
    plt.xlabel("month index")
    plt.ylabel("oil rate (r_oil_s)")
    plt.legend()
    fig_path = os.path.join(out_dir, f"{title.replace(' ','_').lower()}_{idx}.png")
    plt.savefig(fig_path, dpi=140, bbox_inches="tight")
    plt.close()
    return fig_path

# случайные 3 примера из обучаемых скважин
rng = np.random.default_rng(43)
I = np.where(np.isfinite(Y_pred_knn).all(axis=1))[0]
show = rng.choice(I, size=min(3, len(I)), replace=False) if len(I) else []
example_imgs = []
for i in show:
    example_imgs.append(plot_example(i, "knn_example", Y_suffix_true, Y_pred_knn))
    example_imgs.append(plot_example(i, "enet_example", Y_suffix_true, Y_pred_lr))

# 6.8. Простой HTML отчёт
html = f"""
<html><head><meta charset='utf-8'><title>Forecast Report</title></head><body>
<h2>Forecast evaluation (prefix {T_pref} → total {T})</h2>
<p>Generated: {datetime.utcnow().isoformat()}Z</p>
<table border='1' cellspacing='0' cellpadding='6'>
<tr><th>Model</th><th>RMSE</th><th>sMAPE</th><th>N eval wells</th></tr>
<tr><td>KNN</td><td>{m_knn['rmse']:.4f}</td><td>{m_knn['smape']:.4f}</td><td>{m_knn['n_eval']}</td></tr>
<tr><td>ElasticNet</td><td>{m_lr['rmse']:.4f}</td><td>{m_lr['smape']:.4f}</td><td>{m_lr['n_eval']}</td></tr>
</table>
<h3>Files</h3>
<ul>
  <li>metrics.csv</li>
  <li>pred_knn.csv</li>
  <li>pred_elasticnet.csv</li>
</ul>
<h3>Examples</h3>
{''.join(f"<img src='{os.path.basename(p)}' style='max-width:640px;display:block;margin-bottom:10px;'/>" for p in example_imgs)}
</body></html>
"""
report_path = os.path.join(out_dir, "forecast_report.html")
with open(report_path, "w", encoding="utf-8") as f:
    f.write(html)

print("Saved:", metrics_csv, "and", report_path)


KNN   → RMSE=20.3636, sMAPE=0.4884, N=1964
ENet  → RMSE=36.7395, sMAPE=1.1923, N=1964
Saved: ./forecast_exports/metrics.csv and ./forecast_exports/forecast_report.html
